# Find USGS Stations for Manual Evaluation

In [1]:
# This notebook:
# - Connects to the existing `usgs_stations` Qdrant collection
# - Samples surface-water stations by state
# - Checks recent time-series data from USGS Instantaneous Values API
# - Classifies stations as "nice", "messy", or "stale"
# - Suggests candidates for <STATION_ID_A> ... and a reasonable DATE1

import os
import random
from datetime import datetime, timedelta, timezone
from typing import List, Dict, Any, Tuple

import requests
import pandas as pd
from qdrant_client import QdrantClient
from qdrant_client.models import Filter, FieldCondition, MatchValue

# # Qdrant configuration must match ingest_data.py
# QDRANT_HOST = os.getenv("QDRANT_HOST", "qdrant")
# QDRANT_PORT = int(os.getenv("QDRANT_PORT", 6333))
COLLECTION_NAME = "usgs_stations"

# USGS Instantaneous Values API (legacy but stable for now)
# Docs: https://waterservices.usgs.gov/docs/instantaneous-values/instantaneous-values-details/
USGS_IV_URL = "https://waterservices.usgs.gov/nwis/iv/"

# # Connect to Qdrant (same style as ingest_data.py)
# qdrant_client = QdrantClient(
#     host=QDRANT_HOST,
#     port=QDRANT_PORT,
#     grpc_port=6334,
#     prefer_grpc=True,
#     timeout=120,
# )

# collection_info = qdrant_client.get_collection(COLLECTION_NAME)
# print(f"Connected to Qdrant collection '{COLLECTION_NAME}' with "
#       f"{collection_info.points_count} points.")

QDRANT_URL = os.getenv("QDRANT_URL", "http://qdrant:6333")

qdrant_client = QdrantClient(
    url=QDRANT_URL,
    prefer_grpc=False,
    timeout=120
)

collection_info = qdrant_client.get_collection("usgs_stations")
print(
    f"Connected to Qdrant collection 'usgs_stations' with "
    f"{collection_info.points_count} points."
)

def sample_stations_from_qdrant(
    state: str,
    station_type: str = "surface_water",
    sample_size: int = 50,
) -> List[Dict[str, Any]]:
    """
    Sample up to `sample_size` stations from Qdrant for a given state and station_type.

    We use scroll with a payload filter:
    - state == given state (e.g. "AZ")
    - station_type == "surface_water" / "groundwater" / "spring"

    Returns a list of dicts containing payload (station metadata).
    """
    must = [
        FieldCondition(
            key="state",
            match=MatchValue(value=state),
        ),
        FieldCondition(
            key="station_type",
            match=MatchValue(value=station_type),
        ),
    ]
    scroll_filter = Filter(must=must)

    all_points = []
    next_page = None

    # Scroll through the collection
    while True:
        points, next_page = qdrant_client.scroll(
            collection_name=COLLECTION_NAME,
            scroll_filter=scroll_filter,
            limit=64,
            offset=next_page,
            with_payload=True,
            with_vectors=False,
        )
        if not points:
            break
        all_points.extend(points)
        if next_page is None or len(all_points) >= sample_size * 3:
            break

    if not all_points:
        return []

    # Randomly downsample to requested sample_size
    random.shuffle(all_points)
    sample = all_points[:sample_size]

    # Convert to plain dict payloads (keeping id for debugging if desired)
    results = []
    for p in sample:
        payload = dict(p.payload)
        payload["qdrant_id"] = p.id
        results.append(payload)
    return results

def fetch_recent_iv(
    site_id: str,
    days: int = 3,
    parameter_cd: str = "00065",  # gage height
) -> List[Tuple[datetime, float]]:
    """
    Fetch recent instantaneous values for one USGS site.

    Uses the legacy /nwis/iv service:
    https://waterservices.usgs.gov/nwis/iv/?format=json&sites=XXXX&period=P3D&parameterCd=00065 :contentReference[oaicite:4]{index=4}
    """
    params = {
        "format": "json",
        "sites": site_id,
        "period": f"P{days}D",
        "parameterCd": parameter_cd,
    }
    resp = requests.get(USGS_IV_URL, params=params, timeout=30)
    resp.raise_for_status()
    data = resp.json()

    series = data.get("value", {}).get("timeSeries", [])
    if not series:
        return []

    values_blocks = series[0].get("values", [])
    if not values_blocks:
        return []

    values = values_blocks[0].get("value", [])
    records: List[Tuple[datetime, float]] = []

    for row in values:
        ts = row.get("dateTime")
        val = row.get("value")
        if ts is None or val is None:
            continue
        try:
            # USGS timestamps are ISO8601; often end with 'Z'
            t = datetime.fromisoformat(ts.replace("Z", "+00:00"))
            v = float(val)
            records.append((t, v))
        except Exception:
            continue

    return records

def classify_station_timeseries(
    site_id: str,
    days: int = 3,
) -> Dict[str, Any]:
    """
    Classify a station as:
      - "nice": continuous recent data, reasonably dense
      - "messy": some recent data but sparse
      - "stale": last point older than threshold
      - "no_recent_data": no IV data returned

    Also compute basic stats: min/max and fraction of non-zero values.
    """
    records = fetch_recent_iv(site_id, days=days)

    # Default structure so all keys are always present
    ts_info: Dict[str, Any] = {
        "status": "no_recent_data",
        "n_points": 0,
        "age_hours": None,
        "density_pts_per_hour": 0.0,
        "first_ts": None,
        "last_ts": None,
        "min_value": None,
        "max_value": None,
        "nonzero_fraction": 0.0,
    }

    if not records:
        return ts_info

    # Separate timestamps and values
    records.sort(key=lambda r: r[0])
    times = [r[0] for r in records]
    values = [r[1] for r in records]

    now = datetime.now(timezone.utc)
    first_ts, last_ts = times[0], times[-1]
    age_hours = (now - last_ts).total_seconds() / 3600.0
    span_hours = max((last_ts - first_ts).total_seconds() / 3600.0, 1.0)
    n_points = len(values)
    density = n_points / span_hours  # points per hour

    min_val = min(values)
    max_val = max(values)
    nonzero_count = sum(1 for v in values if v > 0.0)
    nonzero_fraction = nonzero_count / n_points if n_points > 0 else 0.0

    # Heuristics for status:
    # - If last point older than 3 days -> "stale"
    # - Otherwise, if density >= 1 pt/hr -> "nice"
    # - Otherwise -> "messy"
    if age_hours > 72:
        status = "stale"
    elif density >= 1.0:
        status = "nice"
    else:
        status = "messy"

    ts_info.update(
        {
            "status": status,
            "n_points": n_points,
            "age_hours": round(age_hours, 2),
            "density_pts_per_hour": round(density, 3),
            "first_ts": first_ts,
            "last_ts": last_ts,
            "min_value": min_val,
            "max_value": max_val,
            "nonzero_fraction": round(nonzero_fraction, 3),
        }
    )
    return ts_info

def build_station_dataframe(
    states: List[str],
    station_type: str = "surface_water",
    sample_per_state: int = 30,
    days: int = 3,
) -> pd.DataFrame:
    """
    For a list of states, sample stations and classify their recent timeseries.
    Returns a pandas DataFrame with station metadata + classification.
    """
    rows = []

    for state in states:
        print(f"Sampling stations for state {state}...")
        samples = sample_stations_from_qdrant(
            state=state,
            station_type=station_type,
            sample_size=sample_per_state,
        )
        print(f"  Got {len(samples)} candidate stations from Qdrant.")

        for s in samples:
            site_id = s.get("station_id")
            if not site_id:
                continue

            ts_info = classify_station_timeseries(site_id, days=days)

            row = {
                "station_id": site_id,
                "station_name": s.get("station_name", ""),
                "station_type": s.get("station_type", ""),
                "state": s.get("state", ""),
                "county": s.get("county", ""),
                "huc_code": s.get("huc_code", ""),
                "latitude": s.get("latitude", None),
                "longitude": s.get("longitude", None),
                "status": ts_info["status"],
                "n_points": ts_info["n_points"],
                "age_hours": ts_info["age_hours"],
                "density_pts_per_hour": ts_info["density_pts_per_hour"],
                "first_ts": ts_info["first_ts"],
                "last_ts": ts_info["last_ts"],
                "min_value": ts_info["min_value"],
                "max_value": ts_info["max_value"],
                "nonzero_fraction": ts_info["nonzero_fraction"],
            }
            rows.append(row)

    df = pd.DataFrame(rows)
    return df

# Choose the states ingested in ingest_data.py
states = ["AZ", "CA", "CO", "NM", "UT", "NV"]

df_stations = build_station_dataframe(
    states=states,
    station_type="surface_water",
    sample_per_state=20,  # you can increase later, but start small
    days=3,
)
print(f"Collected {len(df_stations)} classified stations.")

# Inspect a summary
df_stations["status"].value_counts()


Connected to Qdrant collection 'usgs_stations' with 7426 points.
Sampling stations for state AZ...
  Got 20 candidate stations from Qdrant.
Sampling stations for state CA...
  Got 20 candidate stations from Qdrant.
Sampling stations for state CO...
  Got 20 candidate stations from Qdrant.
Sampling stations for state NM...
  Got 20 candidate stations from Qdrant.
Sampling stations for state UT...
  Got 20 candidate stations from Qdrant.
Sampling stations for state NV...
  Got 20 candidate stations from Qdrant.
Collected 120 classified stations.


status
nice              73
no_recent_data    46
messy              1
Name: count, dtype: int64

## Suggest placeholders for questions.yaml

In [2]:
# Pick:
# - A few nice stations
# - A couple messy/stale ones
# - An upstream/downstream pair (same HUC/state)

nice_df = df_stations[
                      (df_stations["status"] == "nice")
                      & (df_stations["n_points"] >= 50)          # enough points
                      & (df_stations["nonzero_fraction"] >= 0.1) # at least 10% non-zero
                      & (df_stations["max_value"] is not None)
                      & (df_stations["max_value"] > df_stations["min_value"])
                     ].sort_values(
                                   ["state", "nonzero_fraction", "max_value"],
                                   ascending=[True, False, False],
                                  )

messy_df = df_stations[df_stations["status"] == "messy"].sort_values(
                                                                     ["state", "age_hours"],
                                                                     ascending=[True, False],
                                                                    )

stale_df = df_stations[df_stations["status"].isin(["stale", "no_recent_data"])]

print("Nice stations (top 10):")
display(nice_df.head(10))

print("\nMessy stations (top 5):")
display(messy_df.head(5))

print("\nStale / no recent data (top 5):")
display(stale_df.head(5))

def find_upstream_downstream_pair(df: pd.DataFrame) -> pd.DataFrame:
    """
    Heuristic: find two "nice" stations with same state + HUC code.
    This doesn't guarantee true upstream/downstream, but is a usable pair
    for comparison questions.
    """
    grouped = df[df["status"] == "nice"].groupby(["state", "huc_code"])
    candidates = []
    for (state, huc), sub in grouped:
        if not huc or len(sub) < 2:
            continue
        sub_sorted = sub.sort_values("latitude")  # crude proxy for along-river
        if len(sub_sorted) >= 2:
            pair = sub_sorted.head(2)
            candidates.append(pair)
    if not candidates:
        return pd.DataFrame()
    return candidates[0]

up_down = find_upstream_downstream_pair(df_stations)
print("Candidate upstream/downstream pair:")
display(up_down)

# Build a suggestion table for placeholders you can paste into questions.yaml

suggestions = {}

if len(nice_df) >= 2:
    suggestions["STATION_ID_A"] = nice_df.iloc[0]["station_id"]
    suggestions["LOCATION_A"] = f"{nice_df.iloc[0]['station_name']}, {nice_df.iloc[0]['state']}"
    suggestions["STATION_ID_B"] = nice_df.iloc[1]["station_id"]
    suggestions["LOCATION_B"] = f"{nice_df.iloc[1]['station_name']}, {nice_df.iloc[1]['state']}"

if len(messy_df) >= 1:
    suggestions["STATION_ID_D"] = messy_df.iloc[0]["station_id"]
    suggestions["LOCATION_D"] = f"{messy_df.iloc[0]['station_name']}, {messy_df.iloc[0]['state']}"

if not up_down.empty:
    suggestions["STATION_ID_UP"] = up_down.iloc[0]["station_id"]
    suggestions["LOCATION_UP"] = f"{up_down.iloc[0]['station_name']}, {up_down.iloc[0]['state']}"
    suggestions["STATION_ID_DOWN"] = up_down.iloc[1]["station_id"]
    suggestions["LOCATION_DOWN"] = f"{up_down.iloc[1]['station_name']}, {up_down.iloc[1]['state']}"

# Also suggest a DATE1 ~ 3 days ago for custom windows
today_utc = datetime.now(timezone.utc).date()
date1 = today_utc - timedelta(days=3)
suggestions["DATE1"] = date1.isoformat()

suggestions


Nice stations (top 10):


,station_id,station_name,station_type,state,county,huc_code,latitude,longitude,status,n_points,age_hours,density_pts_per_hour,first_ts,last_ts,min_value,max_value,nonzero_fraction
18,09525500,YUMA MAIN CANAL BLW COLORADO R. SIPHON AT YUMA...,surface_water,AZ,,15030108,32.727400,-114.621269,nice,284,1.16,4.014,2025-11-20 20:30:00-07:00,2025-11-23 19:15:00-07:00,10.23,10.97,1.0
3,09480000,"SANTA CRUZ RIVER NEAR LOCHIEL, AZ",surface_water,AZ,,15050301,31.355407,-110.589555,nice,287,0.41,4.014,2025-11-20 20:30:00-07:00,2025-11-23 20:00:00-07:00,6.04,6.08,1.0
13,09505350,"DRY BEAVER CREEK NEAR RIMROCK, AZ",surface_water,AZ,,15060202,34.728631,-111.775708,nice,286,0.66,4.014,2025-11-20 20:30:00-07:00,2025-11-23 19:45:00-07:00,2.67,5.34,1.0
19,09497700,"CIBECUE CREEK NEAR OVERGAARD, AZ",surface_water,AZ,,15060104,34.157222,-110.511111,nice,288,0.16,4.014,2025-11-20 20:30:00-07:00,2025-11-23 20:15:00-07:00,4.70,4.77,1.0
9,09498502,"PINTO CREEK NEAR MIAMI, AZ",surface_water,AZ,,15060103,33.487830,-110.995394,nice,287,0.91,4.042,2025-11-20 20:30:00-07:00,2025-11-23 19:30:00-07:00,1.90,4.73,1.0
6,09499000,"TONTO CREEK ABOVE GUN CREEK, NEAR ROOSEVELT, AZ",surface_water,AZ,,150601050311,33.980054,-111.303484,nice,284,1.16,4.014,2025-11-20 20:30:00-07:00,2025-11-23 19:15:00-07:00,2.41,4.16,1.0
5,09486055,"RILLITO CREEK AT LA CHOLLA BLVD NEAR TUCSON, AZ.",surface_water,AZ,,15050301,32.302761,-111.011375,nice,287,0.41,4.014,2025-11-20 20:30:00-07:00,2025-11-23 20:00:00-07:00,1.82,4.00,1.0
16,09497500,"SALT RIVER NEAR CHRYSOTILE, AZ",surface_water,AZ,,150601030208,33.798119,-110.499849,nice,285,0.91,4.014,2025-11-20 20:30:00-07:00,2025-11-23 19:30:00-07:00,1.66,2.78,1.0
2,09442000,"GILA RIVER NEAR CLIFTON, AZ",surface_water,AZ,,15040002,32.965278,-109.308611,nice,287,0.41,4.014,2025-11-20 20:30:00-07:00,2025-11-23 20:00:00-07:00,2.23,2.41,1.0
17,09485000,"RINCON CREEK NEAR TUCSON, AZ.",surface_water,AZ,,15050302,32.129767,-110.625833,nice,284,1.16,4.014,2025-11-20 20:30:00-07:00,2025-11-23 19:15:00-07:00,1.78,2.41,1.0



Messy stations (top 5):


,station_id,station_name,station_type,state,county,huc_code,latitude,longitude,status,n_points,age_hours,density_pts_per_hour,first_ts,last_ts,min_value,max_value,nonzero_fraction
8,09379025,"CHINLE CREEK AT CHINLE, AZ",surface_water,AZ,,14080204,36.155,-109.5375,messy,71,0.41,0.993,2025-11-20 20:30:00-07:00,2025-11-23 20:00:00-07:00,2.35,2.69,1.0



Stale / no recent data (top 5):


,station_id,station_name,station_type,state,county,huc_code,latitude,longitude,status,n_points,age_hours,density_pts_per_hour,first_ts,last_ts,min_value,max_value,nonzero_fraction
4,09514110,"GILA RIVER BELOW COTTON LN (X2), NEAR GOODYEAR...",surface_water,AZ,,15070101,33.380556,-112.432222,no_recent_data,0,NaN,0.0,None,None,NaN,NaN,0.0
7,09489082,"NORTH FORK THOMAS CREEK NEAR ALPINE, AZ",surface_water,AZ,,15060101,33.675028,-109.270111,no_recent_data,0,NaN,0.0,None,None,NaN,NaN,0.0
10,332747112004501,"GRAND CANAL ABV 32ND STREET AT PHOENIX, AZ",surface_water,AZ,,15060106,33.463000,-112.012472,no_recent_data,0,NaN,0.0,None,None,NaN,NaN,0.0
12,345721111452000,"OAK CREEK ABV SLIDE ROCK STATE PARK NR SEDONA, AZ",surface_water,AZ,,150602020505,34.955844,-111.755506,no_recent_data,0,NaN,0.0,None,None,NaN,NaN,0.0
20,11414265,RUCKER C BL BLUE LK NR EMIGRANT GAP CA,surface_water,CA,,18020125,39.358790,-120.636879,no_recent_data,0,NaN,0.0,None,None,NaN,NaN,0.0


Candidate upstream/downstream pair:


,station_id,station_name,station_type,state,county,huc_code,latitude,longitude,status,n_points,age_hours,density_pts_per_hour,first_ts,last_ts,min_value,max_value,nonzero_fraction
3,09480000,"SANTA CRUZ RIVER NEAR LOCHIEL, AZ",surface_water,AZ,,15050301,31.355407,-110.589555,nice,287,0.41,4.014,2025-11-20 20:30:00-07:00,2025-11-23 20:00:00-07:00,6.04,6.08,1.0
5,09486055,"RILLITO CREEK AT LA CHOLLA BLVD NEAR TUCSON, AZ.",surface_water,AZ,,15050301,32.302761,-111.011375,nice,287,0.41,4.014,2025-11-20 20:30:00-07:00,2025-11-23 20:00:00-07:00,1.82,4.00,1.0


{'STATION_ID_A': '09525500',
 'LOCATION_A': 'YUMA MAIN CANAL BLW COLORADO R. SIPHON AT YUMA, AZ, AZ',
 'STATION_ID_B': '09480000',
 'LOCATION_B': 'SANTA CRUZ RIVER NEAR LOCHIEL, AZ, AZ',
 'STATION_ID_D': '09379025',
 'LOCATION_D': 'CHINLE CREEK AT CHINLE, AZ, AZ',
 'STATION_ID_UP': '09480000',
 'LOCATION_UP': 'SANTA CRUZ RIVER NEAR LOCHIEL, AZ, AZ',
 'STATION_ID_DOWN': '09486055',
 'LOCATION_DOWN': 'RILLITO CREEK AT LA CHOLLA BLVD NEAR TUCSON, AZ., AZ',
 'DATE1': '2025-11-21'}